In [125]:
import requests
import pandas as pd
import asyncio
import time
from typing import Any, Iterable, List, Tuple, Callable
import aiohttp
from bs4 import BeautifulSoup

In [69]:
page = requests.get("https://solicitors.lawsociety.org.uk/person/130606")

In [172]:
page.status_code

200

In [43]:
base_url = "https://solicitors.lawsociety.org.uk/person/"

In [3]:
# first version

df = pd.DataFrame()
for thousand in range(1):
    
    for i in range(thousand*20, (thousand+1)*20):

        async def donwload_aio(base_url, url_indices):
            async def download(base_url, index):
                print(">", end='')
                async with aiohttp.ClientSession() as s:
                    resp = await s.get(base_url+str(index))
                    #data = resp, await resp.text()
                print("<", end='')
                return resp

            return await asyncio.gather(*[download(base_url, index) for index in url_indices])

        result_list = await donwload_aio(base_url, list(range(i*50, (i+1)*50)))

        data = [result.status for result in result_list]
        df_new = pd.DataFrame(data, columns=['status'])
        df = df.append(df_new)
        print('- '+str((i+1)*50))
        time.sleep(1.5)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 50
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 100
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 150
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 200
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 250
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 300
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 350
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 400


Unclosed connection
client_connection: Connection<ConnectionKey(host='solicitors.lawsociety.org.uk', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='solicitors.lawsociety.org.uk', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='solicitors.lawsociety.org.uk', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='solicitors.lawsociety.org.uk', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='solicitors.lawsociety.org.uk', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<Conne

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 450
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 500
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 550
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 600
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 650
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 700
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 750
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 800
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<- 850
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [74]:
async def fetch(session, url):
    async with session.get(url) as response:
        return response.status, await response.text()

async def main(url):
    async with aiohttp.ClientSession() as session:
        return await fetch(session, url)
        #print(status, html)

In [75]:
result = await main(base_url+str(2))
result

(200,
 '<!DOCTYPE html>\r\n<html lang="en-GB">\r\n<head>\r\n    <meta charset="utf-8" />\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\r\n    <title>John Michael Andrews - The Law Society</title>\r\n    <meta name="description">\r\n    <meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1">\r\n\r\n    <script type="text/javascript" src="/Content/scripts/plugins/selectivizr-min.js"></script>\r\n\r\n    <link href="/bundles/all.css?v=sbmINVYoD5EspzthN7d_kS-XWNJ7sIvFLwXAMIC1CXk1" rel="stylesheet" media="all" />\r\n\r\n    <link href="/bundles/print.css?v=slwSdHk4uUBIq6o7BWa0V9Q20GTrfjjSvHywBkgWP9Q1" rel="stylesheet" media="print" />\r\n\r\n    <!--[if lt IE 9]>\r\n        <script src="/Content/scripts/plugins/respond.min.js"></script>\r\n        <script src="/Content/scripts/plugins/html5shiv.js"></script>\r\n    <![endif]-->\r\n    \r\n    \r\n\r\n\r\n    <script>\r\n    \r\n    (function(G,o,O,g,l){G.GoogleAnalyticsObject=O;G[O]||(G[O]

In [76]:
async def load_list(base_url, url_indices):
    return await asyncio.gather(*[main(base_url+str(index)) for index in url_indices])

In [165]:
# second_version

df = pd.DataFrame()

range_start = 55300
range_end = 200000

batch_size = 50
batch_start = int(range_start/batch_size)
batch_end = int(range_end/batch_size)


for batch_index in range(batch_start, batch_end):
    
    result_list = await load_list(base_url, list(range(batch_index*batch_size, (batch_index+1)*batch_size)))
    
    df_new = pd.DataFrame(result_list, columns=['status', 'text'])
    
    print((batch_index+1)*batch_size, end=', ')
    
    df = df.append(df_new, ignore_index=True)
    time.sleep(0.5)

55350, 55400, 55450, 55500, 55550, 55600, 55650, 55700, 55750, 55800, 55850, 55900, 55950, 56000, 56050, 56100, 56150, 56200, 56250, 56300, 56350, 56400, 56450, 56500, 56550, 56600, 56650, 56700, 56750, 56800, 56850, 56900, 56950, 57000, 57050, 57100, 57150, 57200, 57250, 57300, 57350, 57400, 57450, 57500, 57550, 57600, 57650, 57700, 57750, 57800, 57850, 57900, 57950, 58000, 58050, 58100, 58150, 58200, 58250, 58300, 58350, 58400, 58450, 58500, 58550, 58600, 58650, 58700, 58750, 58800, 58850, 58900, 58950, 59000, 59050, 59100, 59150, 59200, 59250, 59300, 59350, 59400, 59450, 59500, 59550, 59600, 59650, 59700, 59750, 59800, 59850, 59900, 59950, 60000, 60050, 60100, 60150, 60200, 60250, 

ClientConnectorError: Cannot connect to host solicitors.lawsociety.org.uk:443 ssl:default [nodename nor servname provided, or not known]

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4950 entries, 0 to 4949
Data columns (total 2 columns):
status    4950 non-null int64
text      4950 non-null object
dtypes: int64(1), object(1)
memory usage: 77.5+ KB


In [169]:
df_200_batch_two = df.loc[df.status == 200]

In [170]:
df_200_batch_two['language_spoken'] = df_200_batch_two.text.apply(language_spoken)

/Users/codyyang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [171]:
df_200_batch_two['language_spoken'][:90000].str.split('\n').apply(lambda x: ('Chinese' in x) or ('Mandarin' in x) or ('Cantonese' in x)).value_counts()

False    2466
True       31
Name: language_spoken, dtype: int64

In [158]:
df_batch_one_553 = df

In [100]:
df_status_200 = df.loc[df.status == 200]

In [101]:
df_status_200.text

2        <!DOCTYPE html>\r\n<html lang="en-GB">\r\n<hea...
5        <!DOCTYPE html>\r\n<html lang="en-GB">\r\n<hea...
7        <!DOCTYPE html>\r\n<html lang="en-GB">\r\n<hea...
8        <!DOCTYPE html>\r\n<html lang="en-GB">\r\n<hea...
9        <!DOCTYPE html>\r\n<html lang="en-GB">\r\n<hea...
                               ...                        
55287    <!DOCTYPE html>\r\n<html lang="en-GB">\r\n<hea...
55288    <!DOCTYPE html>\r\n<html lang="en-GB">\r\n<hea...
55289    <!DOCTYPE html>\r\n<html lang="en-GB">\r\n<hea...
55290    <!DOCTYPE html>\r\n<html lang="en-GB">\r\n<hea...
55292    <!DOCTYPE html>\r\n<html lang="en-GB">\r\n<hea...
Name: text, Length: 29227, dtype: object

In [119]:
df_status_200.text = df_status_200.text.astype(str) 

/Users/codyyang/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [135]:
def language_spoken(item_text):
    soup = BeautifulSoup(item_text, 'html.parser')
    language_spoken = soup.find('div', id="languages-spoken-accordion").get_text().strip()
    return language_spoken

In [136]:
df_status_200['language_spoken'] = df_status_200.text.apply(language_spoken)

/Users/codyyang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [155]:
df_status_200['language_spoken'][:90000].str.split('\n').apply(lambda x: ('Chinese' in x) or ('Mandarin' in x) or ('Cantonese' in x)).value_counts()

False    28864
True       363
Name: language_spoken, dtype: int64

In [126]:
soup = BeautifulSoup(df_status_200.text[2], 'html.parser')

In [134]:
soup.find('div', id="languages-spoken-accordion").get_text().strip()

'English'

In [123]:
df_status_200.to_csv('./df_0_553.csv')

KeyboardInterrupt: 

In [99]:
df_status_200.to_pickle('./df_0_553.csv')

KeyboardInterrupt: 

In [94]:
df.to_csv('./df_0_553.csv')

KeyboardInterrupt: 

In [90]:
df.to_pickle('./df_0_553.pkl')

KeyboardInterrupt: 

In [86]:
df.status.value_counts()

200    29227
404    26073
Name: status, dtype: int64

In [6]:
result_list

[<ClientResponse(https://solicitors.lawsociety.org.uk/person/0) [404 Not Found]>
 <CIMultiDictProxy('Server': 'Microsoft-IIS/8.0', 'Vary': '*', 'Cache-Control': 'public, no-cache="Set-Cookie", max-age=3600', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 15 Apr 2020 01:56:30 GMT', 'Expires': 'Wed, 15 Apr 2020 02:56:30 GMT', 'E-TAG': 'IE=Edge,chrome=1', 'Set-Cookie': 'X-Mapping-fjhppofk=FF2414E2F929F6BD664BBAABF302CD83; path=/', 'Set-Cookie': 'ASP.NET_SessionId=1pdpn0hzlrcdw2zunuyzbaig; path=/; HttpOnly', 'Last-Modified': 'Wed, 15 Apr 2020 01:56:30 GMT', 'Content-Length': '58668')>,
 <ClientResponse(https://solicitors.lawsociety.org.uk/person/1) [404 Not Found]>
 <CIMultiDictProxy('Server': 'Microsoft-IIS/8.0', 'Vary': '*', 'Cache-Control': 'public, no-cache="Set-Cookie", max-age=3600', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 15 Apr 2020 01:56:30 GMT', 'Expires': 'Wed, 15 Apr 2020 02:56:30 GMT', 'E-TAG': 'IE=Edge,chrome=1', 'Set-Cookie': 'X-Mapping-fjhppofk=

In [50]:
data = [result.status for result in result_list]

In [51]:
data

[404, 404, 200, 404, 404, 200, 404, 200, 200, 200]

In [52]:
pd.DataFrame(data, columns=['status'])

,status
0,404
1,404
2,200
3,404
4,404
5,200
6,404
7,200
8,200
9,200


In [26]:
for result in result_list:
    print(result)

(0, 404)
(1, 404)
(2, 200)
(3, 404)
(4, 404)
(5, 200)
(6, 404)
(7, 200)
(8, 200)
(9, 200)


In [41]:
df_summary = pd.DataFrame()

for index in range(3000, 4000):
    
    page = requests.get("https://solicitors.lawsociety.org.uk/person/"+str(index))
    data = {'index': index, 'status': page.status_code}
    df_new = pd.DataFrame.from_records([data])
    df_summary = df_summary.append(df_new) 
    print('-', end='')

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [42]:
df_summary.status.value_counts()

200    529
404    471
Name: status, dtype: int64

In [43]:
df = df.append(df_summary)

In [45]:
df.status.value_counts()

200.0    2107
404.0    1893
Name: status, dtype: int64

In [33]:
soup.find('div', id='languages-spoken-accordion').get_text().strip()

'English'